In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

In [80]:
store = FeatureStore(repo_path="../feature_pipeline/feature_store/feature_repo")

In [81]:
entity_sql = f"""
    SELECT
        parent_asin,
        timestamp as event_timestamp
    FROM {store.get_data_source("parent_asin_rating_stats_source").get_table_query_string()}
    WHERE timestamp BETWEEN '2019-01-01' and '2021-12-31'
"""
training_df = store.get_historical_features(
    entity_df=entity_sql,
    features=[
        "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d",
    ],
).to_df()

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/feast/infra/utils/postgres/connection_utils.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [82]:
training_df

,parent_asin,event_timestamp,parent_asin_rating_cnt_365d
0,B00ASLSQHK,2019-06-18 19:14:47.758,0
1,B003FVYXY0,2019-02-20 19:45:07.082,4
2,B00LT0ZWJE,2019-04-24 13:56:55.072,4
3,B00RXEWOAA,2019-08-30 21:06:05.171,6
4,B00HNFPZF0,2019-08-09 15:35:42.549,2
...,...,...,...
26249,B014UWIKX2,2021-03-22 23:43:29.604,1
26250,B01LW71IBJ,2020-02-29 02:11:23.770,5
26251,B01MS1PMML,2019-01-16 05:30:29.053,6
26252,B08F1P3BCC,2021-02-09 20:51:29.752,42


In [83]:
# get online features

features = store.get_online_features(
    features=[
        "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d",
    ],
    entity_rows=[
        {
            "parent_asin": "B00RXEWOAA",
        }
    ],
).to_dict()

In [84]:
features

{'parent_asin': ['B00RXEWOAA'], 'parent_asin_rating_cnt_365d': [None]}

In [85]:
# get online features

features = store.get_online_features(
    features=[
        "user_rating_stats_fresh:user_rating_list_10_recent_asin",
        "parent_asin_rating_stats_fresh:parent_asin_rating_cnt_365d"
        ""
    ],
    entity_rows=[
        {
            "user_id": "B00RXEWOAA",
            "parent_asin": "B00RXEWOAA",
        }
    ],
).to_dict()

In [86]:
features

{'user_id': ['B00RXEWOAA'],
 'parent_asin': ['B00RXEWOAA'],
 'user_rating_list_10_recent_asin': [None],
 'parent_asin_rating_cnt_365d': [None]}

In [87]:
from datetime import timezone

datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S")

'2025-06-29T08:55:50'

In [88]:
store.materialize_incremental(
    end_date=datetime(2022, 10, 2, tzinfo=timezone.utc))

Materializing 2 feature views to 2022-10-02 07:00:00+07:00 into the postgres online store.

parent_asin_rating_stats_fresh from 2025-06-28 15:55:52+07:00 to 2022-10-02 07:00:00+07:00:


0it [00:00, ?it/s]


user_rating_stats_fresh from 2025-06-28 15:56:03+07:00 to 2022-10-02 07:00:00+07:00:


0it [00:00, ?it/s]
